In [3]:
import argparse
import os
import vindent_utils.utils as utils
import json
import time
import requests

AAI_API_KEY = os.getenv("AAI_API_KEY")

def transcribe(audio_file, question_id="test", user_id="test"):
    if AAI_API_KEY is None:
        raise RuntimeError("AAI_API_KEY environment variable not set. Try setting it now.")

    # Create header with authorization along with content-type
    header = {
        'authorization': AAI_API_KEY,
        'content-type': 'application/json'
    }

    upload_url = utils.upload_file(audio_file, header)

    # Request a transcription
    transcript_response = utils.request_transcript(upload_url, header)

    # Create a polling endpoint that will let us check when the transcription is complete
    polling_endpoint = utils.make_polling_endpoint(transcript_response)

    # Wait until the transcription is complete
    while True:
        polling_response = requests.get(polling_endpoint, headers=header)
        polling_response = polling_response.json()

        if polling_response['status'] == 'completed':
            break

        time.sleep(5)
    paragraphs = utils.get_paragraphs(polling_endpoint, header)

    # Save and print transcript
    with open('transcript.json', 'w') as f:
        f = json.dumps(polling_response)
    
    with open(f'transcript_{question_id}_{user_id}.txt', 'w') as f:
        for para in paragraphs:
            f.write(para['text'] + '\n')
    return paragraphs


ModuleNotFoundError: No module named 'vindent_utils.utils'

In [2]:
import os

AAI_API_KEY = os.getenv("AAI_API_KEY")
print(AAI_API_KEY)

None


In [4]:
header = {
        'authorization': AAI_API_KEY,
        'content-type': 'application/json'
    }

In [5]:
transcript_response, polling_endpoint = transcribe('audio.mp3')

KeyboardInterrupt: 

In [6]:
paragraphs = utils.get_paragraphs(polling_endpoint, header)

In [7]:
for p in paragraphs:
    print(p['text'])

AssemblyAI is a deep learning company that builds powerful APIs to help you transcribe and understand audio. The most common use case for the API is to automatically convert prerecorded audio and video files, as well as real time audio streams into text transcriptions. Our APIs convert audio and video into text using powerful deep learning models that we research and develop end to end in house. Millions of podcasts, zoom recordings, phone calls, or video files are being transcribed with AssemblyAI every single day. But where AssemblyAI really excels is with helping you understand your data.
So let's say we transcribe Joe Biden's State of the Union using assembly. AI's API. With our Auto Chapter feature, you can generate time coded summaries of the key moments of your audio file. For example, with the State of the Union address, we get chapter summaries like this auto Chapters automatically segments your audio or video files into chapters and provides a summary for each of these chapte

In [8]:
import cohere
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
co = cohere.Client(COHERE_API_KEY)

In [47]:
from nltk.tokenize import sent_tokenize
def create_embeds(texts, user_id="test"):
    texts = sent_tokenize(texts)
    embeds = co.embed(texts, model="large", truncate="right")
    return texts, embeds
texts = " ".join([p["text"] for p in paragraphs])
texts, embeds = create_embeds(texts)

In [10]:
len(embeds.embeddings)

23

In [65]:
from transformers import pipeline
from scipy.spatial.distance import cosine
import numpy as np
classifier = pipeline("zero-shot-classification",
                      model="valhalla/distilbart-mnli-12-1")
built_ins = ["entrepreneurship", "confidence", "Growth Mindset"]

def score_BART_text(text, model_name):
    pred = classifier(text, [model_name, "not " + model_name ])
    if pred["scores"][0] > pred["scores"][1] and pred["labels"][0] == model_name:
        return 1
    elif pred["scores"][1] > pred["scores"][0] and pred["labels"][1] == model_name:
        return 1
    else:
        return 0

def score_CUSTOM_text(text_embed, custom_model_embeds):
    mean_score = np.mean([cosine(e, text_embed) for e in custom_model_embeds.embeddings])
    if len(custom_model_embeds.embeddings) < 5:
        threshold = 1.0 - len(custom_model_embeds.embeddings) / 10
    else:
        threshold = 0.5
    if mean_score > threshold:
        return 1
    else:
        return 0
    

In [70]:
import pandas as pd
prefilled_models = {
    "Craftpersonship": "Find meaning in what we do through crafting excellence.",
    "Playfulness": "Great ideas come from health and happiness.",
    "Grit": "Perseverance driven by determination and passion.",
    "Empathy": "Innovation starts with understanding.",
    "Zest": "What sets you apart makes us unique.",
    "Courage": "Dare often and greatly."
}
# Proper customizabiltity would allow for testing and custom thresholds set by the user
# Should there be text highlighting? like themes?
texts_df = pd.DataFrame({"text":texts, "embeddings": embeds.embeddings})

In [74]:
def score_custom_models(texts_df, customs_models=prefilled_models):
    for model_name in custom_models:
        _, embeds_custom = create_embeds(custom_models[model_name], user_id="test")
        texts_df[model_name] = texts_df["embeddings"].apply(lambda x: score_CUSTOM_text(x, embeds_custom))
        time.sleep(10) #Throttle since we're using the free tier for cohere
    return texts_df

In [75]:
texts_df = score_custom_models(texts_df)
for model in built_ins:
     texts_df[model] = score_BART_text(texts_df["text"].values, model)

In [78]:
for _, row in texts_df.iterrows():
    print(row.text)
    print(row)
    print('-'*10)
# In demo, include that a good description for a custom label is 2-3 sentences.

AssemblyAI is a deep learning company that builds powerful APIs to help you transcribe and understand audio.
text                AssemblyAI is a deep learning company that bui...
embeddings          [1.3828125, 0.28051758, 1.9667969, -1.0039062,...
entrepreneurship                                                    0
confidence                                                          0
Growth Mindset                                                      0
craftpersonship                                                     0
Playfulness                                                         1
Grit                                                                0
Empathy                                                             0
Zest                                                                0
Courage                                                             0
Name: 0, dtype: object
----------
The most common use case for the API is to automatically convert prerecorded audio and 

In [83]:
def audio_pipeline(audio_file, question_id="test", user_id="test"):
    texts = transcribe(audio_file, question_id="test", user_id="test")
    texts = " ".join([p["text"] for p in paragraphs])
    texts, embeds = create_embeds(texts)
    texts_df = pd.DataFrame({"text":texts, "embeddings": embeds.embeddings})
    texts_df = score_custom_models(texts_df)
    for model in built_ins:
         texts_df[model] = score_BART_text(texts_df["text"].values, model)
    texts_df.to_csv(f"results/analysis_{question_id}_{user_id}.csv")
    return f"results/analysis_{question_id}_{user_id}.csv"

In [85]:
audio_pipeline("audio.mp3", question_id="test", user_id="test")

AssemblyAI is a deep learning company that builds powerful APIs to help you transcribe and understand audio. The most common use case for the API is to automatically convert prerecorded audio and video files, as well as real time audio streams into text transcriptions. Our APIs convert audio and video into text using powerful deep learning models that we research and develop end to end in house. Millions of podcasts, zoom recordings, phone calls, or video files are being transcribed with AssemblyAI every single day. But where AssemblyAI really excels is with helping you understand your data.

So let's say we transcribe Joe Biden's State of the Union using assembly. AI's API. With our Auto Chapter feature, you can generate time coded summaries of the key moments of your audio file. For example, with the State of the Union address, we get chapter summaries like this auto Chapters automatically segments your audio or video files into chapters and provides a summary for each of these chapt

'results/analysis_test_test.csv'

In [86]:
pd.read_csv("results/analysis_test_test.csv")

,Unnamed: 0,text,embeddings,craftpersonship,Playfulness,Grit,Empathy,Zest,Courage,entrepreneurship,confidence,Growth Mindset
0,0,AssemblyAI is a deep learning company that bui...,"[1.3828125, 0.28051758, 1.9667969, -1.0039062,...",0,1,0,0,0,0,0,0,0
1,1,The most common use case for the API is to aut...,"[0.35839844, 1.578125, 2.1542969, 0.91064453, ...",1,1,1,1,1,1,0,0,0
2,2,Our APIs convert audio and video into text usi...,"[0.7441406, 0.6464844, 1.5029297, -1.5751953, ...",0,0,0,0,0,0,0,0,0
3,3,"Millions of podcasts, zoom recordings, phone c...","[0.24597168, 1.2382812, 2.2871094, 0.37695312,...",0,0,1,0,0,0,0,0,0
4,4,But where AssemblyAI really excels is with hel...,"[1.0498047, 1.7324219, 1.7011719, -0.0770874, ...",0,0,0,0,0,0,0,0,0
5,5,So let's say we transcribe Joe Biden's State o...,"[0.5878906, 0.7602539, 2.7304688, -1.0117188, ...",0,0,0,0,0,0,0,0,0
6,6,AI's API.,"[1.9296875, -0.060272217, 2.015625, -0.0375976...",0,0,0,0,0,0,0,0,0
7,7,"With our Auto Chapter feature, you can generat...","[-0.5488281, 0.45288086, 3.8046875, -0.8676758...",0,1,1,0,0,1,0,0,0
8,8,"For example, with the State of the Union addre...","[-1.5595703, 1.6357422, 2.6601562, 0.11035156,...",1,1,1,1,1,1,0,0,0
9,9,"With sentiment analysis, we can classify what'...","[-1.6367188, 2.0351562, 3.2734375, 0.7006836, ...",0,0,0,0,0,0,0,0,0


In [1]:
from analysis_pipeline import audio_pipeline

C:\Users\liamc\anaconda3\envs\hack\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
audio_pipeline("audio.mp3", question_id="test", user_id="test")

'results/analysis_test_test.csv'

In [5]:
import pandas as pd
pd.read_csv("results/analysis_test_test.csv", index_col=0)
## Use okta for account registration and email confirmation

,text,embeddings,Craftpersonship,Playfulness,Grit,Empathy,Zest,Courage,entrepreneurship,confidence,Growth Mindset
0,AssemblyAI is a deep learning company that bui...,"[1.3828125, 0.28051758, 1.9667969, -1.0039062,...",0,1,0,0,0,0,0,0,0
1,The most common use case for the API is to aut...,"[0.35839844, 1.578125, 2.1542969, 0.91064453, ...",1,1,1,1,1,1,0,0,0
2,Our APIs convert audio and video into text usi...,"[0.7441406, 0.6464844, 1.5029297, -1.5751953, ...",0,0,0,0,0,0,0,0,0
3,"Millions of podcasts, zoom recordings, phone c...","[0.24597168, 1.2382812, 2.2871094, 0.37695312,...",0,0,1,0,0,0,0,0,0
4,But where AssemblyAI really excels is with hel...,"[1.0498047, 1.7324219, 1.7011719, -0.0770874, ...",0,0,0,0,0,0,0,0,0
5,So let's say we transcribe Joe Biden's State o...,"[0.5878906, 0.7602539, 2.7304688, -1.0117188, ...",0,0,0,0,0,0,0,0,0
6,AI's API.,"[1.9296875, -0.060272217, 2.015625, -0.0375976...",0,0,0,0,0,0,0,0,0
7,"With our Auto Chapter feature, you can generat...","[-0.5488281, 0.45288086, 3.8046875, -0.8676758...",0,1,1,0,0,1,0,0,0
8,"For example, with the State of the Union addre...","[-1.5595703, 1.6357422, 2.6601562, 0.11035156,...",1,1,1,1,1,1,0,0,0
9,"With sentiment analysis, we can classify what'...","[-1.6367188, 2.0351562, 3.2734375, 0.7006836, ...",0,0,0,0,0,0,0,0,0


In [22]:
import pandas as pd
BUILT_IN_CUSTOM_MODELS = {
    "Craftpersonship": "Find meaning in what we do through crafting excellence.",
    "Playfulness": "Great ideas come from health and happiness.",
    "Grit": "Perseverance driven by determination and passion.",
    "Empathy": "Innovation starts with understanding.",
    "Zest": "What sets you apart makes us unique.",
    "Courage": "Dare often and greatly."
}
t = pd.DataFrame(BUILT_IN_CUSTOM_MODELS, index=[0]).transpose().reset_index().rename(columns={"index":"custom_model_name", 0:"custom_model_description"})

In [26]:
pd.concat([t, pd.DataFrame({"custom_model_name": "test", "custom_model_description":"cheese"}, index=[0])])

,custom_model_name,custom_model_description
0,Craftpersonship,Find meaning in what we do through crafting ex...
1,Playfulness,Great ideas come from health and happiness.
2,Grit,Perseverance driven by determination and passion.
3,Empathy,Innovation starts with understanding.
4,Zest,What sets you apart makes us unique.
5,Courage,Dare often and greatly.
0,test,cheese
